# MNIST Classifier with HuoguoML

This short notebook shows how to use HuoguoML to deploy and manage a service. This includes:

1. Building and training a neural network to classify MNIST images
2. Create a HuoguoML service

## Requirements

We will be using Tensorflow 2 and HuoguoML. Update `pip` and install packages:

In [ ]:
!pip install --upgrade pip
!pip install tensorflow
!pip install huoguoml

Import all packages:

In [ ]:
import tensorflow as tf
import huoguoml

## Building and training a neural network to classify MNIST images

### Launch HuoguoML server
Launch a HuoguoML server on `127.0.0.1:8080`, where all files are stored to the current directory 

In [ ]:
!huoguoml server --host 127.0.0.1 --port 8080 --artifact_dir=./

### Start a experiment run

In [ ]:
run = huoguoml.start_experiment_run("mnist", "127.0.0.1:8080")

### Data

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Further, the samples are converted from integers to floating-point numbers:

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

run.log_tag("framework", "tensorflow")
run.log_tag("git", "5123131241ed")

### Create a neural network
A two layer fully connected neural network, that is trained with cross entropy loss and optimized with adam model:

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

### Train, evaluate and export neural network

Train neural network

In [ ]:
model.fit(x_train, y_train, epochs=5)

Evaluate neural network -the image classifier should have a accuracy of ~98% on this dataset.

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

Export neural network

In [ ]:
model_dir = "./model"
model.save(model_dir)

## Create a HuoguoML service

In [1]:
import sys
sys.path.insert(0,'..')
import huoguoml
import time
import random

In [2]:
batch_size = [16, 32]
learning_rate = [0.01, 0.002, 0.0005]
momentum = [0.8, 0.9]
random_seed = [42, 23, 2]


train_accuracy = [random.randint(80, 95) for _ in range(5)]
test_accuracy = [random.randint(75, 90) for _ in range(5)]


dataset = ["mnist"]
dataset_version = ["v1", "v2"]


In [4]:
import itertools
i = 0
for element in itertools.product(*[batch_size, learning_rate, momentum, random_seed, train_accuracy, test_accuracy, dataset, dataset_version]):
    bs, lr, m, rs, tra, tsa, d, dv = element
    
    with huoguoml.start_experiment_run(d, "http://127.0.0.1:8080") as run:
        run.log_parameter("batch_size", bs)
        run.log_parameter("learning_rate", lr)
        run.log_parameter("momentum",m)
        run.log_parameter("random_seed", rs)        
        run.log_parameter("optimizer", "SGD")
        
        run.log_metric("train_accuracy", tra)
        run.log_metric("test_accuracy", tsa)
        
        run.log_tag("dataset", d)
        run.log_tag("dataset_version", dv)
        run.log_model("tensorflow", tf_saved_model_dir="./model",
                            tf_meta_graph_tags="serve",
                            tf_signature_def_key="serving_default")
    i = i+1
    if i == 6:
        break